In [2]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt


In [4]:
# simple van der Pol example
# assume that coeff are the estimated coefficients
coeff = np.zeros(19) # 1 noise, 
    # x, y, xx, xy, yy, xxx, xxy, xyy, yyy for dx/dt and dy/dt
coeff[0] = 0.05 # noise
coeff[1] = 0.11
coeff[2] = 1.1  #true
coeff[4] = -0.15 
coeff[10] = -0.95 #true
coeff[11] = 1.9 #true
coeff[14] = 0.12
coeff[16] = -2.1 #true

In [5]:
# approximate range of values
xmin = -2
xmax = 2
ymin = -4
ymax = 4

In [7]:
x, y = symbols("x y")

In [10]:
yy = y**2


 2
y 


In [14]:
coeff[1:].reshape(2,-1)[0]

array([ 0.11,  1.1 ,  0.  , -0.15,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ])

In [22]:
# compute the Drift of x
D1x = Integer(0)
sigma_x = (coeff[1:].reshape(2,-1))[0]
Poly = list([x, y])
Dim = 2
CoeffIndex = 0

for i in range(Dim): # linear
    print(Poly[i])
    D1x += sigma_x[CoeffIndex]*Poly[i]
    CoeffIndex +=1

for i in range(Dim): # quadratic
    for j in range(i,Dim):
        print(Poly[i]*Poly[j])
        D1x += sigma_x[CoeffIndex]*Poly[i]*Poly[j]
        CoeffIndex +=1
for i in range(Dim): # cubic
    for j in range(i,Dim):
        for k in range(j,Dim):
            print(Poly[i]*Poly[j]*Poly[k])
            D1x += sigma_x[CoeffIndex]*Poly[i]*Poly[j]*Poly[k]
            CoeffIndex +=1

x
y
x**2
x*y
y**2
x**3
x**2*y
x*y**2
y**3


In [23]:
D1x

-0.15*x*y + 0.11*x + 1.1*y

In [26]:
# compute the Drift of x
D1y = Integer(0)
sigma_y = (coeff[1:].reshape(2,-1))[1]
Poly = list([x, y])
Dim = 2
CoeffIndex = 0

for i in range(Dim): # linear
    print(Poly[i])
    D1y += sigma_y[CoeffIndex]*Poly[i]
    CoeffIndex +=1

for i in range(Dim): # quadratic
    for j in range(i,Dim):
        print(Poly[i]*Poly[j])
        D1y += sigma_y[CoeffIndex]*Poly[i]*Poly[j]
        CoeffIndex +=1
for i in range(Dim): # cubic
    for j in range(i,Dim):
        for k in range(j,Dim):
            print(Poly[i]*Poly[j]*Poly[k])
            D1y += sigma_y[CoeffIndex]*Poly[i]*Poly[j]*Poly[k]
            CoeffIndex +=1
D1y.simplify()

x
y
x**2
x*y
y**2
x**3
x**2*y
x*y**2
y**3


-2.1*x**2*y - 0.95*x + 0.12*y**2 + 1.9*y

In [27]:
len(Poly)

2

In [28]:
def GetPoly(List, Coeff, Constant = False):
    # takes List of (linear) base functions [x1, x2, x3,..] and 
    # their polynomial coefficients Coeff of third order
    # returns a sympy polynomial
    # if Constant = True: the first Coeff[0] is used as constant
    
    Out = Integer(0)
    Index = 0 # iterating over the Coefficients
    Dim = len(List)
    
    if Constant:
        Out += Coeff[Index]
        Index +=1
    
    for i in range(Dim): # linear
        Out += Coeff[Index] * List[i]
        Index +=1
    
    for i in range(Dim): # quadratic
        for j in range(i,Dim):
            Out +=Coeff[Index] * List[i] * List[j]
            Index +=1
            
    for i in range(Dim): # cubic
        for j in range(i,Dim):
            for k in range(j,Dim):
                Out += Coeff[Index] * List[i] * List[j] * List[k]
                Index +=1
    return(Out)

In [37]:
Polyx = GetPoly(Poly, sigma_x)
Polyx

-0.15*x*y + 0.11*x + 1.1*y

In [34]:
Polyx.subs({x:1, y:2})

2.01000000000000

In [35]:
integrate(Polyx,
          (x, xmin, xmax), 
          (y, ymin,ymax))

0

In [40]:
m_Polyx_x =  integrate(Polyx, # maringalised over x
                      (x, xmin, xmax)) / (xmax-xmin)
m_Polyx_y =  integrate(Polyx, # maringalised over y
                      (y, ymin, ymax)) / (ymax-ymin)

In [41]:
m_Polyx_x, m_Polyx_y

(1.1*y, 0.11*x)

In [44]:
(Polyx - m_Polyx_x)**2

0.0225*(-x*y + 0.733333333333333*x)**2

In [48]:
Causal_Polyx_x = integrate((Polyx - m_Polyx_x)**2, 
                          (x, xmin, xmax),
                           (y, ymin, ymax))
Causal_Polyx_y = integrate((Polyx - m_Polyx_y)**2, 
                          (x, xmin, xmax),
                           (y, ymin, ymax))

In [49]:
print(Causal_Polyx_x, Causal_Polyx_y)

5.63626666666667 211.626666666667


In [50]:
(xmax-xmin)*(ymax-ymin)

32